In [1]:
# !pip install evaluate
# !pip install scikit-learn
# !pip install trl

# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

In [2]:
## PEFT examples: https://github.com/huggingface/peft/tree/main/examples

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [3]:
import torch
import numpy as np

import evaluate
from evaluate import evaluator
from datasets import load_dataset
from torch.utils.data import DataLoader
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, DataCollatorWithPadding
from sklearn.model_selection import train_test_split

In [4]:
MODEL_NAME = "distilbert-base-uncased"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128

In [5]:
dataset = load_dataset("sms_spam", split="train")
dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=42)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [7]:
def tokenize(examples):
    # return_tensors="pt" ensures that the tokenized output is in pytorch tensors
    # truncation=True ensures that all input into the model has consistent size.
    # Padded/truncated to the max_length of the model

    return tokenizer(examples["sms"], padding="max_length",
                     truncation=True, return_tensors="pt")

def compute_metrics(eval_pred):
    """
    Wrapper method to do calculation for metrics that we are interested in
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    eval_loss = torch.nn.CrossEntropyLoss()(torch.tensor(logits), torch.tensor(labels))
    metrics = metric.compute(predictions=predictions, references=labels)
    metrics['eval_loss'] = eval_loss.item()
    return metrics

In [8]:
metric = evaluate.load("accuracy", "cross_entropy")

In [9]:
tokenized_dataset = dataset.map(tokenize, batched=True)

In [10]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2,
                                                          id2label={0: "not spam", 1: "spam"},
                                                          label2id={"not spam": 0, "spam": 1})

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

In [12]:
# Create a Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [13]:
trainer.evaluate()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6388779282569885,
 'eval_accuracy': 0.8609865470852018,
 'eval_runtime': 19.4052,
 'eval_samples_per_second': 57.459,
 'eval_steps_per_second': 0.464}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [14]:
## Specifying the task_type will then create the relevant instance of the object
## Reference: https://github.com/huggingface/peft/blob/02ae6bcb373d9d9d3bec9ba920d63316418ff64a/src/peft/peft_model.py#L1094C7-L1094C41

## Available task types: https://huggingface.co/docs/peft/en/package_reference/peft_types
lora_config = LoraConfig(task_type="SEQ_CLS", r=4, lora_alpha=1,
                         lora_dropout=0, target_modules=["pre_classifier", "classifier"],
                         inference_mode=False)

In [15]:
peft_lora_model = get_peft_model(model, lora_config)
peft_lora_model.print_trainable_parameters()

trainable params: 1,202,708 || all params: 67,565,588 || trainable%: 1.7800599914856066


In [16]:
peft_lora_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): DistilBertForSequenceClassification(
      (distilbert): DistilBertModel(
        (embeddings): Embeddings(
          (word_embeddings): Embedding(30522, 768, padding_idx=0)
          (position_embeddings): Embedding(512, 768)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (transformer): Transformer(
          (layer): ModuleList(
            (0-5): 6 x TransformerBlock(
              (attention): MultiHeadSelfAttention(
                (dropout): Dropout(p=0.1, inplace=False)
                (q_lin): Linear(in_features=768, out_features=768, bias=True)
                (k_lin): Linear(in_features=768, out_features=768, bias=True)
                (v_lin): Linear(in_features=768, out_features=768, bias=True)
                (out_lin): Linear(in_features=768, out_features=768, bias=True)
              )
             

In [22]:
# Define training arguments
peft_training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=15,
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=16,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
#     eval_steps=5,
#     logging_steps=5,
    save_strategy='epoch',
    save_safetensors=False,
    ## https://github.com/huggingface/transformers/issues/27613#issuecomment-1848645557

    warmup_steps=500,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",


    # Can play with remove_unused_columns. Initially set this to False because Trainer is returning
    # IndexError: Invalid key: 4437 is out of bounds for size 0
    # https://discuss.huggingface.co/t/indexerror-invalid-key-16-is-out-of-bounds-for-size-0/14298/4
    remove_unused_columns=False,
#     label_names=['label']

    # By default, Trainer uses GPU on the device
    # However, if you want to explicitly set GPU device(s)
    # no_cuda=False,  # Set to False to enable GPU usage
    # device=[0],  # Use GPU device with index 0, in case you have multiple GPUs
)

In [23]:
# Create a Trainer instance
peft_trainer = Trainer(
    model=peft_lora_model,
    args=peft_training_args,

    # We are dropping the SMS column because the size of this input column is not consistent
    # Not removing this column will lead to
    # ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`sms` in this case) have excessive nesting (inputs type `list` where type `int` is expected).
    train_dataset=tokenized_dataset["train"].remove_columns(["sms"]),
    eval_dataset=tokenized_dataset["test"].remove_columns(["sms"]),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
#.rename_column("label", "labels")

In [19]:
## https://stackoverflow.com/questions/76012700/validation-loss-shows-no-log-during-fine-tuning-model

In [ ]:
# Train the model
peft_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.627100,0.615822,0.866368
2,0.605300,0.582866,0.866368
3,0.565500,0.533437,0.866368
4,0.512700,0.474258,0.866368
5,0.452500,0.412958,0.866368
6,0.395600,0.358087,0.866368
7,0.344900,0.315554,0.866368


In [ ]:
# Save the trained model
peft_trainer.save_model(f"./results/{MODEL_NAME}-best")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [ ]:
output_dir = f"./results/{MODEL_NAME}-best"

In [ ]:
loaded_model = AutoModelForSequenceClassification.from_pretrained(output_dir)

In [ ]:

saved_tokenizer = AutoTokenizer.from_pretrained(output_dir)
saved_model = AutoModelForSequenceClassification.from_pretrained(output_dir, load_in_4bit=True, device_map="auto")